In [1]:
import pandas as pd
df = pd.read_csv("Data_Marketing_Customer_Analysis_Round3.csv")


### DATA WRANGLING

In [2]:
df = df.dropna()
df = df.drop_duplicates()
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10497 entries, 0 to 10688
Data columns (total 24 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   region                         10497 non-null  object
 1   customer_lifetime_value        10497 non-null  int64 
 2   response                       10497 non-null  object
 3   coverage                       10497 non-null  object
 4   education                      10497 non-null  object
 5   effective_to_date              10497 non-null  object
 6   month                          10497 non-null  object
 7   employment_status              10497 non-null  object
 8   gender                         10497 non-null  object
 9   income                         10497 non-null  int64 
 10  location_code                  10497 non-null  object
 11  marital_status                 10497 non-null  object
 12  monthly_premium_auto           10497 non-null  int64 
 13  months

In [3]:
numericals.head()

NameError: name 'numericals' is not defined

### SPLIT

In [ ]:
 # Defining x explanatory variables and 1x y traget variable

In [ ]:
X = df[["customer_lifetime_value","income","monthly_premium_auto","months_since_last_claim","months_since_policy_inception",
        "number_of_open_complaints","number_of_policies"]]
y = df[["total_claim_amount"]]

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state = 100)

### SCALING WITH PT (PowerTransformer)

In [ ]:
# SCALE VARIABLES AFTER THE SPLIT (NOT TO MIX TEST AND TRAIN DATA)

In [ ]:
import seaborn as sns

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler, PowerTransformer
import numpy as np

In [ ]:
# Iniciated the PowerTransformer my my case # "pt is the scaler 
pt = PowerTransformer()

# Fit the transformer to my X_train data 
pt = pt.fit(X_train)

# Transform X_train using the fitted transformer
X_train_trans = pt.transform(X_train)

# Transform X_test using the fitted transformer
X_test_trans = pt.transform(X_test) 

# keeps them separately. It also ensures consistency in the transformation process, but only for X! 

# --------------------WHY DONT WE SCALE Y???? --------------------------------

In [ ]:
y_train.hist()

###  ALTERNATIVE: SCALING WITH STANDARD SCALER 

In [ ]:
# calling the scaler and saving with scaler 
#scaler = StandardScaler()

# create the frame/fit
#scaler = scaler.fit(X_train)

# populate with data/transform and save in new variables 
#X_train_trans = scaler.transform(X_train)
#X_test_trans = scaler.transform(X_test)

### Model training

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
# i train the model with the transformed training sets from exploring var x and **non-transformed** labels y
lm = LinearRegression()
model = lm.fit(X_train_trans,y_train) # fitting the model to the data! using the liner functon formula. 

#creates the linear function line

# y_train can be scaled by not in this task. 

# calculating the line of best fit 
# the process my test data goes through must be the same train data. e.g. same scaler 

In [ ]:
model.coef_

In [ ]:
model.intercept_

# --------- intercept for what? ---------------- >> the best intercept at the y axe for this linear model

### Model Interpretation

In [ ]:
predictions = model.predict(X_test_trans)
predictions

In [ ]:
# -----  is this correct???? do i use the unscaled x_test? why? ----------

random_test = X_test.sample()

In [ ]:
random_test

In [ ]:
model.predict(random_test)

### Create predictions for test set

In [ ]:
y_pred = model.predict(X_test_trans)

In [ ]:
y_pred = pd.DataFrame(y_pred)

In [ ]:
y_test = y_test.reset_index(drop=True)

In [ ]:
resiudals_df = pd.concat([y_test,y_pred],axis=1)

In [ ]:
residuals_df = resiudals_df.rename(columns={"total_claim_amount":"y_test", 0:"y_pred"})

# -------- WHAT IS HAPPENING HERE? --------  NOT ENTIERLY CLEAR ----------

In [ ]:
residuals_df["residual"] = residuals_df["y_test"]-residuals_df["y_pred"]

In [ ]:
round(residuals_df,2)

### Calculating metrics

In [ ]:
mean_error = residuals_df["residual"].mean()

In [ ]:
round(mean_error,2)

In [ ]:
from sklearn.metrics import mean_squared_error as mse , mean_absolute_error as mae

In [ ]:
round(mse(y_test,y_pred),2)

In [ ]:
round(mae(y_test,y_pred),2)


In [ ]:
rmse = mse(y_test,y_pred, squared = False)
round(rmse,2)

## Lab instructions 31.01.2024:

### - Compute R2 for the first model iteration + feature importance plot
### - Run the 2nd iteration of the model, adding the categorical data

### Computing R2_score

In [ ]:
from sklearn.metrics import r2_score

#### 1/2 Computing R2_score for test_set

In [ ]:
round(r2_score(y_test,y_pred),2)

#### 2/2 Computing R2_score for TRAIN_set

In [ ]:
y_pred_train = pd.DataFrame(model.predict(X_train_trans))

In [ ]:
y_pred_train

In [ ]:
round(r2_score(y_train,y_pred_train),2)

In [ ]:
round(r2_score(X_train,y_pred),2)
# ERROR HERE!

In [ ]:
feature_names = list(X_train.columns)
feature_names

coefficients = list(model.coef_)
coefficients

coefficients = [i for i in coefficients[0]]

fi_dict = {"name": feature_names, "coeff":coefficients}
fi_df = pd.DataFrame(fi_dict)
fi_df

In [ ]:
sns.barplot(x = fi_df["name"], y = fi_df["coeff"])
# here we compare the importance of the variables.

#---- ONLY WORKS WHEN ALL FOLLOW THE SAME SCALE -----

In [ ]:
categoricals = ca_df.select_dtypes(include=object)
numericals = ca_df.select_dtypes(include = np.number)

In [ ]:
ordinals = categoricals[["coverage","education","vehicle_size"]]
nominals = categoricals.drop(columns=["coverage","education","vehicle_size","effective_to_date"])

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

In [ ]:
ordinals["coverage"]

In [ ]:
enc = OrdinalEncoder(categories=[["basic","extended","premium"]])
ordinals["coverage"] = enc.fit_transform(ordinals[["coverage"]])

enc = OrdinalEncoder(categories=[["college", "bachelor", "high school or below", "doctor", "master"]])
ordinals["education"] = enc.fit_transform(ordinals[["education"]])

enc = OrdinalEncoder(categories=[["medsize", "small", "large"]])
ordinals["vehicle_size"] = enc.fit_transform(ordinals[["vehicle_size"]])

In [ ]:
ordinals

In [ ]:
nominals = pd.get_dummies(nominals, dtype=int)

In [ ]:
nominals

In [ ]:
categorical_coded = pd.concat([ordinals,nominals], axis=1)
categorical_coded

In [ ]:
categorical_coded.isnull().sum()

In [ ]:
final_df = pd.concat([numericals,categorical_coded], axis=1)
final_df.isnull().sum()


In [ ]:
final_df.columns


### SPLIT

In [ ]:
X = final_df.drop(columns = ["total_claim_amount"])

y = final_df[["total_claim_amount"]]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state = 100)